# Project #2

## 1. Multifactor Model
Run a multifactor model fot the stock and portfolio created in Project 1.\
The factors are:

    a) Market
    b) SMB
    c) HML
    d) TERM: difference in yields between 10 years and 3 months U.S. Treasuries
    e) CREDIT: difference in yields between BAA and AAA rated U.S. corporate bonds

GUIDE: [medium.com](https://medium.com/swlh/how-to-build-a-multi-factor-equity-portfolio-in-python-4560fab3df7b)

# Project1 - Formula
$$x_t = \alpha + \underbrace{\beta (R_m - R_f) + \varepsilon}_{\text 1 factor}$$

# Project2 - Formula
$$x_t = \alpha + \beta 
\begin{bmatrix}
f_1 & \\
f_2 & \\
\dots{} & \\
f_n 
\end{bmatrix}
+ \varepsilon$$



### 1.1. Methodology

In the literature, there are two different ways to build a multi-factor portfolio, *bottom-up* or *top-down*. Top-down combines targeted factor portfolios, think 'combining silos together', and bottom-up ranks each security on their overall factor rank and chooses the securities that have the best overall score of all the factors. All of the literature I have come across favors the **bottom-up** approach, probably due to the fact you may end up duplicating trades in the top-down approach and potentially have better diversification with a blended signal.

In this notebook I decided to use a bottom-up blended signal approach in building the Python script, with the aim of gaining exposure to the following factors:
+ Market
+ SMB: Small Minus Big
+ HML: High Minus Low
+ TERM: difference in yields between 10 years and 3 months U.S. Treasuries

In [2]:
# IMPORTS
import datetime as dt
import pandas as pd
import pandas_datareader as pdr

In [3]:
# RISK-FREE RETURN FAMA&FRENCH
dateparse = lambda x: dt.datetime.strptime(x,'%Y%m')
french_fries_df = pd.read_csv('./data/F-F_Research_Data_Factors.CSV',
                        header=0,
                        names=['Date','Mkt-RF','SMB','HML','RF'],
                        parse_dates=['Date'], date_parser=dateparse,
                        index_col=0,
                        skipfooter=99,
                        skiprows=3,
                        engine='python')
ff_3_monthly = french_fries_df.loc[french_fries_df.index >= '2000-11-1']
# Rf
risk_free_rets = french_fries_df['RF'].loc[french_fries_df.index >= '2000-01-01']/100
# (Rm - Rf)
#market_risk_premium = french_fries_df['Mkt-RF'].loc[french_fries_df.index >= '2000-01-01']/100
#french_fries_df.loc[french_fries_df.index == '1993-01-01']

In [24]:
# GET STOCK ASSET
ticker = 'AAPL'
start = dt.datetime(2000,11,1)
end = dt.datetime(2022,1,1)

stock_df = pdr.get_data_yahoo(ticker, start, end, interval='m')
stock_df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-11-01,0.410714,0.287946,0.347098,0.294643,8.488374e+09,0.252295
2000-12-01,0.312500,0.243304,0.303571,0.265625,8.858970e+09,0.227448
2001-01-01,0.401786,0.257813,0.265625,0.386161,1.370946e+10,0.330660
2001-02-01,0.391741,0.321429,0.369420,0.325893,7.023766e+09,0.279054
2001-03-01,0.424107,0.306920,0.318080,0.394107,1.079906e+10,0.337464
...,...,...,...,...,...,...
2021-09-01,157.259995,141.270004,152.830002,141.500000,1.797835e+09,141.113998
2021-10-01,153.169998,138.270004,141.899994,149.800003,1.565079e+09,149.391357
2021-11-01,165.699997,147.479996,148.990005,165.300003,1.691029e+09,164.849091


In [27]:
# GET STOCK RETURNS
stock_returns = stock_df['Adj Close'].pct_change().dropna()
stock_returns

Date
2000-12-01   -0.098485
2001-01-01    0.453782
2001-02-01   -0.156070
2001-03-01    0.209315
2001-04-01    0.154963
                ...   
2021-09-01   -0.066640
2021-10-01    0.058657
2021-11-01    0.103471
2021-12-01    0.075796
2022-01-01   -0.015712
Name: Adj Close, Length: 254, dtype: float64

In [29]:
ff_data = pd.merge(ff_3_monthly, stock_returns, left_index=True, right_index=True)

ff_data

,Mkt-RF,SMB,HML,RF,Adj Close
Date,,,,,
2000-12-01,1.19,0.73,7.60,0.50,-0.098485
2001-01-01,3.13,6.68,-5.11,0.54,0.453782
2001-02-01,-10.05,-0.74,12.48,0.38,-0.156070
2001-03-01,-7.26,0.27,6.43,0.42,0.209315
2001-04-01,7.94,0.56,-4.69,0.39,0.154963
...,...,...,...,...,...
2021-09-01,-4.37,0.80,5.09,0.00,-0.066640
2021-10-01,6.65,-2.28,-0.44,0.00,0.058657
2021-11-01,-1.55,-1.35,-0.53,0.00,0.103471
